<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/330-add-a-nl-direction-field/50_ode/50_Chaotic_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nl



In [ ]:
import ode_solver



# Simulation of ODE Systems<br>상미방 시스템의 수치 시뮬레이션
## The Lorenz System.<br>로렌츠 계



references :
* https://en.wikipedia.org/wiki/Lorenz_system
* https://behance.net/gallery/7618879/Strange-Attractors
* https://en.wikipedia.org/wiki/List_of_chaotic_maps
* https://matplotlib.org/stable/gallery/mplot3d/scatter3d.html
* https://gemini.google.com



We can use ODE solvers to calculate the trajectories of chaotic systems.<br>
ODE solver 는 카오스 시스템의 궤적을 계산하기 위해서도 사용할 수 있다.



In 1963, Prof. Edward Lorenz and his colleagues developed a simplifed mathematical model of atmospheric convection.<br>1963년 미국의 기상학자이자 수학자인 에드워드 로렌츠가 동료들과 함께 대기의 대류 현상을 수학적으로 단순화하여 표현하기 위해 개발한 모델이다.



The Lorenz attractor is a set of chaotic solutions; an attractor is a region in the state space where a system tends to evolve towards over time, often regardless of its initial conditions within a particular region.<br>
Lorenz Attractor 로렌츠 끌개란 그 카오스 해의 집합을 말한다. Attractor 끌개 란, 시간이 흐름에 따라 시스템이 상태 공간상 특정 영역 안에서 시작하였을 때 도달하게 되는 상태공간상의 영역 또는 평형점을 말한다.



$$
\begin{align}
    \frac{dx}{dt} & = \sigma (y - x) \\
    \frac{dy}{dt} & = x (\rho - z) - y \\
    \frac{dz}{dt} & = x y - \beta z
\end{align}
$$


The Lorenz system models a simplified scenario of a two-dimensional fluid layer uniformly warmed below and cooled from above.<br>해당 미분방정식이 나타내는 로렌츠 계는 아래에서 덥혀지고 위에서 냉각되는 유체의 움직임에 관한 것이다.



Please consider trying different values of the following parameters of the Lorenz system<br>아래 로렌츠 계 매개변수의 다른 값을 시도해 보는 것도 생각해 보자.



In [ ]:
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0



In [ ]:
def get_Lorenz_system_slope(sigma:float, rho:float, beta:float):
    '''
    Return a closure calculating slope function
    기울기 함수를 계산하는 내포 함수를 반환
    '''
    def f(t:float, xv:np.ndarray,):
        '''
        Calculate slope vector of the Lorenz system
        로렌츠 계의 기울기 벡터를 계산
        '''
        x, y, z = xv
        dx_dt = sigma * (y - x)
        dy_dt = x * (rho - z) - y
        dz_dt = x * y - beta * z

        return np.array((dx_dt, dy_dt, dz_dt))

    return f


Lorenz_attractor = get_Lorenz_system_slope(sigma=sigma, rho=rho, beta=beta)



In [ ]:
def sim_attractor(
        attractor, x_0:np.ndarray=(np.array((1, 1, 1)) * 1.0),
        sigma:float=10.0, rho:float=28.0, beta:float=(8.0/3.0),
        t_end=20.0,
        elev_deg:float=None, azim_deg:float=None, figsize=(14, 14)
    ):
    '''
    Simulate a chatoic attractor
    카오스 계 해 곡선을 시뮬레이션
    '''
    t_array = np.arange(0, t_end, 1e-3)

    t_out, x_out = ode_solver.rk4(attractor, t_array, x_0)
    x_out = np.array(x_out)
    ax = plt.figure(figsize=figsize).add_subplot(projection="3d")
    ax.view_init(elev_deg, azim_deg)
    ax.plot(x_out[:, 0], x_out[:, 1], x_out[:, 2])
    ax.plot(
        (x_out[0, 0], x_out[-1, 0]),
        (x_out[0, 1], x_out[-1, 1]),
        (x_out[0, 2], x_out[-1, 2]),
        'o'
    )
    ax.text(x_out[0, 0], x_out[0, 1], x_out[0, 2], 'start')
    ax.text(x_out[-1, 0], x_out[-1, 1], x_out[-1, 2], 'end')

    ax.set_xlabel('rate of convection')
    ax.set_ylabel('horizontal temperature variation')
    ax.set_zlabel('vertical temperature variation')
    ax.grid(True)

    return ax, t_out, x_out



In [ ]:
%time ax_0, t_out_0, x_out_0 = sim_attractor(Lorenz_attractor, (np.array((1, 1, 1)) * 1.0))



Now let's slightly change the initial condition. How different is the final point this time?<br>이제 초기 조건을 살짝 바꾸어 보자. 마지막 점의 위치는 얼마 정도 달라졌는가?



In [ ]:
%time ax_1, t_out_1, x_out_1 = sim_attractor(Lorenz_attractor, (np.array((1, 1, 1.1)) * 1.0))



Distances between two trajectories from two initial conditions.<br>
두 초기 조건으로부터의 궤적(`x_out_0` vs `x_out_1`)의 각 싯점(`t_out_0[i]`)에서의 차이를 계산해 보자.



In [ ]:
delta_x = x_out_1 - x_out_0
dist_x = nl.norm(delta_x, axis=1)
dist_x.shape



The plot will be as follows.<br>그래프는 다음과 같을 것이다.



In [ ]:
plt.semilogy(t_out_0, dist_x)
plt.xlabel('time')
plt.ylabel('distance')
plt.grid(True)



To analyze such nonlinear systems, how can numerical methods such as RK4 be helpful?<br>이러한 비선형 시스템 해석에 RK4와 같은 수치 해법이 어떤 도움이 될 수 있다고 생각하는가?

